In [43]:
import nsepy as nse
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:


# Function to create a DataFrame with stock metrics
def create_stock_metrics_df(stock_dict):
    data = []
    for stock_name, ticker_symbol in stock_dict.items():
        ticker = yf.Ticker(ticker_symbol)
        metrics = {
            'Stock': stock_name,
            'Ticker': ticker_symbol,
            'PE Ratio': ticker.info.get('trailingPE', 'N/A'),
            'EPS': ticker.info.get('trailingEps', 'N/A'),
            'Dividend Yield': ticker.info.get('dividendYield', 'N/A'),
            'Market Cap': ticker.info.get('marketCap', 'N/A'),
            '52-Week High': ticker.info.get('fiftyTwoWeekHigh', 'N/A'),
            '52-Week Low': ticker.info.get('fiftyTwoWeekLow', 'N/A'),
            '90-Day Moving Average': ticker.history(period="3mo")['Close'].mean(),
            '180-Day Moving Average': ticker.history(period="6mo")['Close'].mean(),
            'Volume': ticker.history(period="1d")['Volume'].sum()
            #'Debt-to-Equity Ratio': 'N/A'  # Placeholder, calculation needs to be done differently
        }
        data.append(metrics)
    df = pd.DataFrame(data)
    df.replace('N/A', pd.NA, inplace=True)
    return df

# Function to plot stock metrics using Plotly
def plot_stock_metrics(df, metric):
    fig = px.bar(df, x='Stock', y=metric, title=f'Stock {metric}', labels={'Stock': 'Stock', metric: metric})
    fig.show()


def plot_combination_chart(df):
    metrics_columns = df.columns[2:]  # Exclude 'Stock' and 'Ticker' columns
    normalized_df = df.copy()

    # Normalize the data (Min-Max Scaling)
    for metric in metrics_columns:
        min_val = df[metric].min()
        max_val = df[metric].max()
        normalized_df[metric] = (df[metric] - min_val) / (max_val - min_val)

    fig = go.Figure()

    for metric in metrics_columns:
        sorted_df = normalized_df.sort_values(by='Stock', ascending=True)
        fig.add_trace(go.Bar(
            x=sorted_df['Stock'],
            y=sorted_df[metric],
            name=metric
        ))

    fig.update_layout(
        barmode='group',  # Group bars side by side
        title='Normalized Stock Metrics for Nifty 50 Stocks',
        xaxis_title='Stock',
        yaxis_title='Normalized Metric Value',
        height=600,
        width=1200,
    )
    
    fig.show()




In [45]:
# Function to plot data for a given ticker
def plot_ticker(data, ticker_name,cols_to_plot=["Open", "High", "Low", "Close"],common_axis="Datetime"):
    # Creating a figure with subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Adding traces for the specified columns on the primary y-axis
    for col in cols_to_plot:
        fig.add_trace(go.Scatter(x=data[common_axis], y=data[col], mode='lines', name=col), secondary_y=False)

    # Adding a trace for the volume on the secondary y-axis
    fig.add_trace(go.Scatter(x=data[common_axis], y=data['Volume'], mode='lines', name='Volume'), secondary_y=True)

    # Updating the layout
    fig.update_layout(title=f'{ticker_name} Intraday Price Movement',
                      xaxis_title='Time')

    # Updating y-axis titles
    fig.update_yaxes(title_text='Price (INR)', secondary_y=False)
    fig.update_yaxes(title_text='Volume', secondary_y=True)

    # Displaying the plot
    fig.show()

In [46]:
nse_tickers = {
    "Tata_Motors": "TATAMOTORS.NS",
    # "NSE": "^NSEI",
    "Reliance_Industries": "RELIANCE.NS",
    "HDFC_Bank": "HDFCBANK.NS",
    "Infosys": "INFY.NS",
    "ICICI_Bank": "ICICIBANK.NS",
    "Hindustan_Unilever": "HINDUNILVR.NS",
    # "Kotak_Mahindra_Bank": "KOTAKBANK.NS",
    # "State_Bank_of_India": "SBIN.NS",
    # "Larsen_Toubro": "LT.NS",
    # "Axis_Bank": "AXISBANK.NS",
    # "Bajaj_Finance": "BAJFINANCE.NS",
    # "HCL_Technologies": "HCLTECH.NS",
    # "Maruti_Suzuki": "MARUTI.NS",
    # "Tata_Steel": "TATASTEEL.NS",
    # "Sun_Pharma": "SUNPHARMA.NS",
    # "NTPC": "NTPC.NS"
}



In [47]:


# Fetch historical data and store in a dictionary
stock_data_dict     = {ticker: yf.Ticker(nse_tickers[ticker]).history(period="1d", interval="1m").reset_index() for ticker in nse_tickers}

# Create DataFrame 
df = create_stock_metrics_df(nse_tickers) 


#Plot data for each ticker
for ticker_name, data in stock_data_dict.items():
    plot_ticker(data, ticker_name)

# Plot a specific metric 
#plot_stock_metrics(df, 'Volume')
plot_all_stock_metrics(df)
plot_combination_chart(df)

In [48]:
ticker = yf.Ticker("TATAMOTORS.NS")
display(ticker.info)  # Company information
display(ticker.financials)  # Financial statements
display(ticker.balance_sheet)  # Balance sheet
display(ticker.cashflow)  # Cash flow statement
df

{'address1': 'Ahura Centre',
 'address2': '4th Floor 82 Mahakali Caves Road, MIDC Andheri (East)',
 'city': 'Mumbai',
 'zip': '400 093',
 'country': 'India',
 'phone': '91 22 6240 7101',
 'website': 'https://www.tatamotors.com',
 'industry': 'Auto Manufacturers',
 'industryKey': 'auto-manufacturers',
 'industryDisp': 'Auto Manufacturers',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Tata Motors Limited designs, develops, manufactures, and sells various automotive vehicles. The company offers passenger cars; sports utility vehicles; intermediate and light commercial vehicles; small, medium, and heavy commercial vehicles; defense vehicles; pickups, wingers, buses, vans, and trucks; and electric vehicles, as well as related spare parts and accessories. It also manufactures engines for industrial applications, and aggregates comprising axles and transmissions for commercial vehicles; and factory automation e

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Tax Effect Of Unusual Items,-6554370000.0,1195840000.0,5479050000.0,-40059450000.0,NaN
Tax Rate For Calcs,0.3,0.2,0.3,0.3,NaN
Normalized EBITDA,673957900000.0,378818500000.0,253177900000.0,327282800000.0,NaN
Total Unusual Items,-21847900000.0,5979200000.0,18263500000.0,-133531500000.0,NaN
Total Unusual Items Excluding Goodwill,-21847900000.0,5979200000.0,18263500000.0,-133531500000.0,NaN
...,...,...,...,...,...
Salaries And Wages,NaN,NaN,308183100000.0,281931800000.0,308372800000.0
Gross Profit,1490464900000.0,1047343500000.0,2440451300000.0,817568900000.0,NaN
Cost Of Revenue,2859376300000.0,2381402400000.0,311901000000.0,1664062500000.0,NaN
Total Revenue,4349841200000.0,3428745900000.0,2752352300000.0,2481631400000.0,NaN


,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Ordinary Shares Number,3832241897.0,3829847221.0,3829055161.0,3828810661.0,NaN
Share Issued,3832241897.0,3829847221.0,3829055161.0,3828810661.0,NaN
Net Debt,610332300000.0,962735300000.0,1027680300000.0,1041937000000.0,NaN
Total Debt,1098104000000.0,1366134500000.0,1476990300000.0,1421197700000.0,NaN
Tangible Book Value,201219100000.0,-112881300000.0,-133122700000.0,-112886800000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,550320400000.0,513764200000.0,634452300000.0,650971900000.0,NaN
Other Short Term Investments,150172800000.0,194894700000.0,252862200000.0,333971800000.0,NaN
Cash And Cash Equivalents,400147600000.0,318869500000.0,381590100000.0,317000100000.0,NaN
Cash Equivalents,332238900000.0,236140600000.0,276806400000.0,316889000000.0,NaN


,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Free Cash Flow,365017300000.0,172923300000.0,-8855900000.0,87951100000.0,NaN
Repayment Of Debt,-504082600000.0,-634175200000.0,-448896100000.0,-342528100000.0,NaN
Issuance Of Debt,218235000000.0,444405100000.0,481748600000.0,504845200000.0,NaN
Issuance Of Capital Stock,818700000.0,37696000000.0,37685900000.0,26025100000.0,NaN
Capital Expenditure,-314136300000.0,-180956800000.0,-151684200000.0,-202054000000.0,NaN
...,...,...,...,...,...
Earnings Losses From Equity Investments,NaN,NaN,740600000.0,3789600000.0,10000000000.0
Gain Loss On Investment Securities,279200000.0,-881700000.0,-2065900000.0,-12718700000.0,NaN
Net Foreign Currency Exchange Gain Loss,-5375500000.0,37466100000.0,-8785100000.0,-43981200000.0,NaN
Gain Loss On Sale Of Business,NaN,NaN,NaN,0.0,0.0


,Stock,Ticker,PE Ratio,EPS,Dividend Yield,Market Cap,52-Week High,52-Week Low,90-Day Moving Average,180-Day Moving Average,Volume
0,Tata_Motors,TATAMOTORS.NS,8.601022,90.03,0.0039,2850475212800,1179.00,717.70,798.988280,912.585432,8678662
1,Reliance_Industries,RELIANCE.NS,25.223063,50.21,0.0040,17138106499072,1608.80,1201.50,1279.824614,1376.062181,14116032
2,HDFC_Bank,HDFCBANK.NS,18.724356,88.23,0.0119,12635374354432,1880.00,1363.55,1759.578905,1707.415356,10605895
3,Infosys,INFY.NS,28.920965,66.68,0.0210,7987697549312,2006.45,1358.35,1890.805189,1874.240249,7527493
4,ICICI_Bank,ICICIBANK.NS,18.891409,66.12,0.0081,8818745409536,1362.35,970.15,1285.063280,1257.052013,11895661
5,Hindustan_Unilever,HINDUNILVR.NS,53.710490,43.66,0.0181,5510257442816,3035.00,2172.05,2454.828812,2625.433021,2762503
